In [1]:
import pydicom
import cv2 as cv2
import numpy as np
import os
import shutil

In [9]:
base_path = os.getcwd()
base_folder = base_path + '\\LGG-DICOM\\'
save_path = base_path + '\\LGG\\'

In [2]:
def delete_non_referenced_dicoms(image_folder, referenced_instances_file):
    print(f"Image folder has {len(os.listdir(image_folder))} dicom files")
    referened_sop_instance_uids = []

    if referenced_instances_file is not None:
        with open(referenced_instances_file, 'r') as f:
            for line in f:
                referened_sop_instance_uids.append(line.strip())   

        count = 0

        # Deleting dicom files that are not referenced
        for file in os.listdir(image_folder):
            file_path = os.path.join(image_folder, file)
            dicom_data = pydicom.dcmread(file_path)
            
            sop_instance_uid = dicom_data.SOPInstanceUID

            if sop_instance_uid not in referened_sop_instance_uids:
                count += 1
                os.remove(file_path)

        print(f"Deleted {count} non-referenced dicom files")

In [18]:
def transform_image_from_3d_to_2d(image_folder, output_folder, flip_y):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    if len(os.listdir(output_folder)) > 0:
        print(f"Output folder already has 2D images")
        return

    file_path = os.listdir(image_folder)[0]
    dicom_data = pydicom.dcmread(os.path.join(image_folder, file_path))
    pixel_array = dicom_data.pixel_array

    print(f'Image resolution: {pixel_array.shape}')

    # Save each slice as an image
    for i in range(pixel_array.shape[0]):
        slice_2d = pixel_array[i, :, :]

        if flip_y:
            slice_2d = np.flipud(slice_2d) 
        
        scaled_img = (slice_2d * 255).astype('uint8')
        
        # Define output path
        output_path = os.path.join(output_folder, f"tumor_slice_{i + 1}.png")
        
        # Save image
        cv2.imwrite(output_path, scaled_img)

    print("All slices have been saved as 2D.")

In [17]:
def save_dicom_as_png(image_folder, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    if len(os.listdir(output_folder)) > 0:
        print(f"Output folder already has 2D images")
        return

    for file in os.listdir(image_folder):  
        file_path = os.path.join(image_folder, file)
        dicom_data = pydicom.dcmread(file_path)
        pixel_array = dicom_data.pixel_array

        # Normalize pixel values
        min_val, max_val = pixel_array.min(), pixel_array.max()
        scaled_image = ((pixel_array - min_val) / (max_val - min_val) * 255).astype('uint8')

        slice_number = file.split('.')[0].split('-')[-1]

        # Define output path
        output_path = os.path.join(output_folder, f"t2_slice_{slice_number}.png")

        # Save image
        cv2.imwrite(output_path, scaled_image)


    print("All dicom files have been saved as 2D.")

In [6]:
# Read flips from file
def read_flips():
    flips_y = {}
    flips_path = base_folder + 'flips.txt'

    with open(flips_path, 'r') as f:
        flips_lines = f.readlines()
        for flip_line in flips_lines:
            patient_number = flip_line.strip().split(' ')[0]
            flip_str = flip_line.strip().split(' ')[1]
            # flip should be a boolean value
            flip = True if flip_str == 'True' else False
            flips_y[patient_number] = flip
            
    return flips_y

In [7]:
def clean_referenced_instances_file(referenced_instances_file):
    with open(referenced_instances_file, 'r') as f:
        lines = f.readlines()
    
    with open(referenced_instances_file, 'w') as f:
        for line in lines:
            # If line contains Class UID, remove it
            if 'Class UID' in line:
                continue
            number_part_uid = line.split('UID')[-1]
            number_part = number_part_uid.split(' ')[-1]
            f.write(number_part)

In [21]:
for folder in os.listdir(base_folder):
    if folder == 'LICENSE' or folder == 'flips.txt':
        continue

    print(folder)
    
    folder_path = base_folder + folder + '\\'
    for folder2 in os.listdir(folder_path):
        if 'Segmentation' in folder2:
            tumor_folder = folder_path + folder2 + '\\'
            output_tumor_folder = save_path + folder + '\\' + folder2 + '\\'
        elif 'T2' in folder2:
            image_folder = folder_path + folder2 + '\\'
            output_image_folder = save_path + folder + '\\' + folder2 + '\\'

    referenced_instances_file = folder_path + "ReferencedSOPInstanceUIDs.txt"

    # # Transforming tumor segmentation from 3D to 2D
    # if os.path.exists(tumor_folder):
    #     flips_y = read_flips()
    #     flip_y_flag = flips_y[folder] if folder in flips_y else False
    #     print(f"Transforming tumor segmentation from 3D to 2D flip flag: {flip_y_flag}")
    #     transform_image_from_3d_to_2d(tumor_folder, output_tumor_folder, flip_y_flag)
    #     print(f"Tumor folder has {len(os.listdir(output_tumor_folder))} files")

    # # Deleting non referenced dicoms
    # if os.path.exists(referenced_instances_file):
    #     print("Cleaning referenced instances file for " + folder)
    #     clean_referenced_instances_file(referenced_instances_file)
    #     print("Deleting non-referenced dicoms from " + image_folder)
    #     delete_non_referenced_dicoms(image_folder, referenced_instances_file)
    # else:
    #     print(f"All images already have a corresponding tumor slice file")

    # Saving dicom files as png
    if os.path.exists(image_folder):
        print("Saving dicom files as png for " + image_folder)
        save_dicom_as_png(image_folder, output_image_folder)

    print()

LGG-104
Saving dicom files as png for c:\Users\masac\OneDrive\Desktop\Masa\College\DIE\Masa\LGG-DICOM\LGG-104\T2\
Output folder already has 2D images

LGG-203
Saving dicom files as png for c:\Users\masac\OneDrive\Desktop\Masa\College\DIE\Masa\LGG-DICOM\LGG-203\T2\
Output folder already has 2D images

LGG-210
Saving dicom files as png for c:\Users\masac\OneDrive\Desktop\Masa\College\DIE\Masa\LGG-DICOM\LGG-210\T2\
Output folder already has 2D images

LGG-216
Saving dicom files as png for c:\Users\masac\OneDrive\Desktop\Masa\College\DIE\Masa\LGG-DICOM\LGG-216\T2\
Output folder already has 2D images

LGG-218
Saving dicom files as png for c:\Users\masac\OneDrive\Desktop\Masa\College\DIE\Masa\LGG-DICOM\LGG-218\T2\
Output folder already has 2D images

LGG-219
Saving dicom files as png for c:\Users\masac\OneDrive\Desktop\Masa\College\DIE\Masa\LGG-DICOM\LGG-219\T2\
Output folder already has 2D images

LGG-220
Saving dicom files as png for c:\Users\masac\OneDrive\Desktop\Masa\College\DIE\Masa\LG

In [ ]:
# Optional script to rename the folders and adhere to predefined fodler structure
for folder in os.listdir(base_folder):
    if folder == 'LICENSE' or folder == 'flips.txt':
        continue

    print(folder)
    target_folder = os.listdir(os.path.join(base_folder, folder))[0]
    case_folder = base_folder + folder + "\\" + target_folder + "\\"
    image_folder = None
    tumor_folder = None
    
    for f in os.listdir(case_folder):
        if 'segmentation' in f.lower():
            tumor_folder = case_folder + f + "\\"
            continue
        if 'sop' in f.lower():
            continue

        image_folder = case_folder + f + "\\"

    if image_folder and tumor_folder:   
        # Rename and move the image folder to the parent level
        new_image_folder = os.path.join(base_folder, folder, 'T2')
        new_tumor_folder = os.path.join(base_folder, folder, 'Segmentation')
        
        # Rename (move) the folders
        os.rename(image_folder, new_image_folder)
        os.rename(tumor_folder, new_tumor_folder)
        
        # Delete the original target folder
        shutil.rmtree(case_folder)  # Removes the target_folder directory and its contents
        
        print(f"Renamed {image_folder} to {new_image_folder}")
        print(f"Renamed {tumor_folder} to {new_tumor_folder}")
        print(f"Deleted original target folder: {case_folder}")


LGG-104
LGG-203
Renamed c:\Users\masac\OneDrive\Desktop\Masa\College\DIE\LGG\LGG-203\08-06-1997-NA-MRI Hd wow-40871\4.000000-Post Gad axial FSE-00285\ to c:\Users\masac\OneDrive\Desktop\Masa\College\DIE\LGG\LGG-203\T2
Renamed c:\Users\masac\OneDrive\Desktop\Masa\College\DIE\LGG\LGG-203\08-06-1997-NA-MRI Hd wow-40871\300.000000-Tumor segmentation-75999\ to c:\Users\masac\OneDrive\Desktop\Masa\College\DIE\LGG\LGG-203\Segmentation
Deleted original target folder: c:\Users\masac\OneDrive\Desktop\Masa\College\DIE\LGG\LGG-203\08-06-1997-NA-MRI Hd wow-40871\
LGG-210
Renamed c:\Users\masac\OneDrive\Desktop\Masa\College\DIE\LGG\LGG-210\12-21-1989-NA-MRI Hd wow-52776\3.000000-AX T2 OBL-21631\ to c:\Users\masac\OneDrive\Desktop\Masa\College\DIE\LGG\LGG-210\T2
Renamed c:\Users\masac\OneDrive\Desktop\Masa\College\DIE\LGG\LGG-210\12-21-1989-NA-MRI Hd wow-52776\300.000000-Tumor segmentation-55952\ to c:\Users\masac\OneDrive\Desktop\Masa\College\DIE\LGG\LGG-210\Segmentation
Deleted original target fold